In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline



Using TensorFlow backend.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
path_file = os.getcwd()+ '/lines/lines-aaa'
text = []
count = 0
with open(path_file ,'r') as txtfile:
  for line in txtfile.readlines():
    if count == 20000:
      break
    if len(line.strip().lower()) > 30:
        text.append(line.strip().lower())
        count +=1
    
    


In [16]:
vocab_size =  20000
max_len = 15
trunc_token = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(text)
padded = pad_sequences(sequences,maxlen=max_len, truncating=trunc_token,padding = 'post')




In [17]:
word_index

{'<OOV>': 1,
 'the': 2,
 'you': 3,
 'to': 4,
 'a': 5,
 'i': 6,
 'and': 7,
 'of': 8,
 'in': 9,
 'it': 10,
 'that': 11,
 'for': 12,
 'me': 13,
 'is': 14,
 'we': 15,
 'be': 16,
 'have': 17,
 'if': 18,
 'your': 19,
 'with': 20,
 'this': 21,
 'he': 22,
 'on': 23,
 'are': 24,
 'but': 25,
 'not': 26,
 'will': 27,
 'my': 28,
 'was': 29,
 'do': 30,
 'like': 31,
 'what': 32,
 'get': 33,
 "don't": 34,
 'him': 35,
 "it's": 36,
 'all': 37,
 'can': 38,
 'as': 39,
 'at': 40,
 "i'm": 41,
 'know': 42,
 'one': 43,
 'when': 44,
 'from': 45,
 'there': 46,
 'they': 47,
 'up': 48,
 'about': 49,
 'his': 50,
 'so': 51,
 'her': 52,
 'here': 53,
 'us': 54,
 'go': 55,
 'our': 56,
 'out': 57,
 "you're": 58,
 'an': 59,
 'just': 60,
 'them': 61,
 'no': 62,
 'now': 63,
 'time': 64,
 'by': 65,
 'or': 66,
 'she': 67,
 'got': 68,
 'man': 69,
 'how': 70,
 "can't": 71,
 'think': 72,
 'take': 73,
 'see': 74,
 'well': 75,
 'why': 76,
 'tell': 77,
 'back': 78,
 'want': 79,
 'who': 80,
 'were': 81,
 "i'll": 82,
 'would': 83,

In [18]:
index_word = {}
for word , index in word_index.items():
    index_word[index] = word
    
    

In [19]:
def create_word_embeddings(file_path):
    
    with open(file_path , 'r') as f:
        wordToEmbedding = {}
        wordToIndex = {}
        indexToWord = {}
        
        for line in f:
            data = line.strip().split()
            token = data[0]
            wordToEmbedding[token] = np.array(data[1:] ,dtype = np.float64)



        
        tokens = sorted(wordToEmbedding.keys())
        for idx , token in enumerate(tokens):
            idx = idx + 1 #for zero masking
            wordToIndex[token] = idx
            indexToWord[idx] = token

    return wordToEmbedding , wordToIndex , indexToWord
            
            

In [20]:
wordToEmbedding , wordToIndex , indexToWord = create_word_embeddings(os.getcwd()+ '/glove.6B/glove.6B.200d.txt')

In [21]:
len(wordToIndex)    

400000

In [22]:
def create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord):
    
    vocablen = 20000 #for zero masking
    embedding_dimensions = 200
    
    embeddingMatrix = np.zeros((vocablen , embedding_dimensions))
    count = 0
    for word , index in word_index.items():
        if word not in wordToEmbedding.keys():
            embeddingMatrix[index ,:] = np.random.uniform(low = 0 , high =1 ,size = (1,200))
            count +=1
        else :
            
            embeddingMatrix[index , :] = wordToEmbedding[word]
        
    embeddingLayer = Embedding(vocablen , embedding_dimensions , weights = [embeddingMatrix] , trainable = False)
    print(embeddingMatrix.shape)
    print(count)
    
    return embeddingLayer

In [23]:
embeddingLayer = create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord)

(20000, 200)
691


In [24]:
batch_size = 10000
context = np.zeros((10000,max_len))
response = np.zeros((10000,max_len))
def get_dataset(padded):
    index = 0
    for idx in range(0,20000,2):
        context[index,:] = padded[idx]
        response[index,:] = padded[idx+1]
        index +=1

In [25]:
get_dataset(padded)

In [26]:
response = list(response.swapaxes(0,1))

In [27]:
padded[5000]

array([  10,  127,   17,   86, 4836,    9,    5, 1101,    8, 2685, 2089,
          8,  120,  328,    0], dtype=int32)

In [28]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [29]:
Tx = 15
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)


In [30]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator(inputs =([a,s_prev]))
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context


In [31]:
post_activation_LSTM_cell = LSTM(128, return_state = True) # post-attention LSTM 
output_layer = Dense(20000, activation=softmax)

In [32]:

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = embeddingLayer(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [33]:
model = model(15, 15, 128, 128, 20000, 20000)


In [34]:
opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

In [35]:
s0 = np.zeros((batch_size, 128))
c0 = np.zeros((batch_size, 128))

In [36]:
model.fit([context, s0, c0], response, epochs=10, batch_size=100)

Epoch 1/10
10000/10000 [==============================] - 55s 5ms/step - loss: 77.6744 - dense_3_loss: 2.1304 - dense_3_accuracy: 0.0000e+00 - dense_3_accuracy_1: 0.0000e+00 - dense_3_accuracy_2: 2.0000e-04 - dense_3_accuracy_3: 0.0013 - dense_3_accuracy_4: 0.0067 - dense_3_accuracy_5: 0.0473 - dense_3_accuracy_6: 0.1667 - dense_3_accuracy_7: 0.3317 - dense_3_accuracy_8: 0.4990 - dense_3_accuracy_9: 0.6174 - dense_3_accuracy_10: 0.7108 - dense_3_accuracy_11: 0.7782 - dense_3_accuracy_12: 0.8292 - dense_3_accuracy_13: 0.8683 - dense_3_accuracy_14: 0.8925
Epoch 2/10
10000/10000 [==============================] - 48s 5ms/step - loss: 69.1323 - dense_3_loss: 1.4110 - dense_3_accuracy: 0.0000e+00 - dense_3_accuracy_1: 0.0000e+00 - dense_3_accuracy_2: 2.0000e-04 - dense_3_accuracy_3: 0.0013 - dense_3_accuracy_4: 0.0067 - dense_3_accuracy_5: 0.0475 - dense_3_accuracy_6: 0.1676 - dense_3_accuracy_7: 0.3347 - dense_3_accuracy_8: 0.5045 - dense_3_accuracy_9: 0.6238 - dense_3_accuracy_10: 0.7180 

In [37]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(20000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [38]:
s = [("how are you ?")]

In [39]:
seq = tokenizer.texts_to_sequences(s)
pad = pad_sequences(seq,maxlen=max_len, truncating=trunc_token,padding = 'post')


In [40]:
pad

array([[70, 24,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [41]:
pred = model.predict([pad , np.zeros((1,128)) , np.zeros((1,128))])

In [42]:
pred

[array([[1.9056967e-02, 2.1528159e-07, 2.5126534e-02, ..., 1.8820150e-07,
         2.3144895e-07, 2.2319277e-07]], dtype=float32),
 array([[4.2869423e-02, 2.3528729e-08, 4.4414263e-02, ..., 1.9620797e-08,
         2.5777897e-08, 2.4842452e-08]], dtype=float32),
 array([[5.7034016e-02, 1.6314807e-08, 4.3902636e-02, ..., 1.3558253e-08,
         1.7877857e-08, 1.7334457e-08]], dtype=float32),
 array([[6.9034368e-02, 1.5592860e-08, 4.3661345e-02, ..., 1.2961081e-08,
         1.7130979e-08, 1.6586108e-08]], dtype=float32),
 array([[8.88023451e-02, 1.56434510e-08, 4.21818644e-02, ...,
         1.30169395e-08, 1.72516348e-08, 1.66706542e-08]], dtype=float32),
 array([[1.2753634e-01, 1.5795116e-08, 3.8945891e-02, ..., 1.3160615e-08,
         1.7519421e-08, 1.6890315e-08]], dtype=float32),
 array([[2.1160217e-01, 1.5460021e-08, 3.2906193e-02, ..., 1.2902384e-08,
         1.7274226e-08, 1.6614361e-08]], dtype=float32),
 array([[3.8852736e-01, 1.3171942e-08, 2.3295751e-02, ..., 1.1014641e-08,
   

In [43]:
pred[1].shape

(1, 20000)

In [44]:
len(pred)

15

In [45]:
ans = beam_search_decoder(pred)

In [46]:
ans

[([3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -20.7835973807456),
 ([6, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -20.88623755029329),
 ([3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -20.89626116078245)]

In [50]:
index_word[3]

'you'

In [52]:
index_word[6]

'i'

In [49]:
ask = [("what is your name")]

seq = tokenizer.texts_to_sequences(ask)
pad = pad_sequences(seq,maxlen=max_len, truncating=trunc_token,padding = 'post')



In [ ]:
pad

In [ ]:
pred = model.predict([pad , np.zeros((1,128)) , np.zeros((1,128))])

In [ ]:
len(pred)

In [ ]:
ans = beam_search_decoder(pred)

In [ ]:
ans
